## GEOSeMOSYS - exploring linear programming geospatial energy modelling 

Nandi Moksnes

This Jupyter Notebook is dectription of how to reproduce the results in the paper:
For any questions please contact Nandi Moksnes: nandi@kth.se

## 1. The files in "GIS_data" are downloaded and placed in a "temp" folder.

In [6]:
from Download_files.py import *

download_url_data(input_data/GIS_URL.txt temp)
unzip_all(input_data/GIS_URL.txt)

download_url_data('input_data/zenodo.txt', '../Projected_files')

## 2. The files are then projected and clipped to the administrative boundaries.

In [1]:
from Project_GIS import *

main('../GIS_data')

## 3. Through QGIS make raster to point layer and save

1) masked_UMT37S_ken_ppp_2015_UNadj: SAGA raster to point 

This step is downloaded from zenodo so it is not needed for the user to execute.


## 4. The GIS layers are prepared to for a heuristic approximation for electrified settlements

In [ ]:
from settlement_build import *

points = raster_to_point(pop_shp, Projected_files_path)
point_line = near_calculations_line(points, Projected_files_path)
settlements = near_calculations_point(point_line, Projected_files_path)
settlements 

## 5. Approximate location of urban settlemtns and the electrified settlements 1kmx1km resolution

In [6]:
from elec_start import *

pop_actual = 52570000 #peolpe
urban = 0.275 #percent
urban_cutoff = 20000
urbansettlements = calibrate_pop_and_urban(settlements, pop_actual, urban, urban_cutoff)

elec_actual = 0.75 #percent
pop_cutoff = 40000 #people
dist_minigrid = 5000 #meters
dist_to_trans = 50000 #meters
min_night_lights = 0.01
max_grid_dist= 200000 #meters
max_road_dist = 300 #meters 
pop_cutoff2 = 360000 #people
urban_elec_ratio=83.5 #percent
rural_elec_ratio= 71.5 #percent

elec_current_and_future = (urbansettlements, elec_actual, pop_cutoff, dist_to_trans, min_night_lights,
                            max_grid_dist, urban_elec_ratio, rural_elec_ratio,  max_road_dist, pop_actual, pop_cutoff2, start_year)


()